In [1]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

In [2]:
monthly_noise = pd.read_csv('..\Data\monthly_noisedata_2022.csv', header = 0, sep=',')
daily_noise = pd.read_csv('..\Data\daily_noisedata_2022.csv', header = 0, sep=',')
hourly_noise = pd.read_csv('..\Data\hourly_noisedata_2022.csv', header = 0, sep=',')

FileNotFoundError: [Errno 2] No such file or directory: '..\\Data\\monthly_noisedata_2022.csv'

In [ ]:
# create a dataframe
gps_data = {
    'description': ['MP 01: Naamsestraat 35  Maxim', 'MP 02: Naamsestraat 57 Xior', 'MP 03: Naamsestraat 62 Taste', 'MP 04: His & Hears', 'MP 05: Calvariekapel KU Leuven', 'MP 06: Parkstraat 2 La Filosovia', 'MP 07: Naamsestraat 81', 'MP08bis - Vrijthof'],
    'lat': [50.877121, 50.87650, 50.87590, 50.875237, 50.87452, 50.874078, 50.873808, 50.87873],
    'lon': [4.700708, 4.700632, 4.700262, 4.700071, 4.69985, 4.70005, 4.700007, 4.70115]
}

gps_df = pd.DataFrame(gps_data)
gps_df.head(10)
  

,description,lat,lon
0,MP 01: Naamsestraat 35 Maxim,50.877121,4.700708
1,MP 02: Naamsestraat 57 Xior,50.876500,4.700632
2,MP 03: Naamsestraat 62 Taste,50.875900,4.700262
3,MP 04: His & Hears,50.875237,4.700071
4,MP 05: Calvariekapel KU Leuven,50.874520,4.699850
5,MP 06: Parkstraat 2 La Filosovia,50.874078,4.700050
6,MP 07: Naamsestraat 81,50.873808,4.700007
7,MP08bis - Vrijthof,50.878730,4.701150


In [ ]:
# merge noise data with gps
merged = pd.merge(daily_noise, gps_df, on='description', how='left')
# Add a new column 'year' with value 2022 for all observations
merged['year'] = 2022

# Create a new 'date' column by combining 'year', 'month', and 'day'
merged['date'] = pd.to_datetime(merged[['year', 'month', 'day']])

reference_date = pd.to_datetime('2022-01-01')
merged['days_since_reference'] = (merged['date'] - reference_date).dt.days

# Create a StandardScaler object
scaler = MinMaxScaler()

# Fit the StandardScaler to the column and transform the lamax values
merged['standardized_lamax'] = scaler.fit_transform(merged[['lamax']])

# Fit the StandardScaler to the column and transform the lamax values
merged['standardized_laeq'] = scaler.fit_transform(merged[['laeq']])

merged.head()

,month,day,description,lamax,laeq,lat,lon,year,date,standardized_lamax,standardized_laeq
0,1,1,MP 03: Naamsestraat 62 Taste,51.665242,49.992637,50.875900,4.700262,2022,2022-01-01,0.515869,0.538979
1,1,1,MP 05: Calvariekapel KU Leuven,48.747476,46.504067,50.874520,4.699850,2022,2022-01-01,0.415931,0.416866
2,1,1,MP 06: Parkstraat 2 La Filosovia,48.270005,46.007220,50.874078,4.700050,2022,2022-01-01,0.399577,0.399474
3,1,1,MP 07: Naamsestraat 81,45.908501,44.373056,50.873808,4.700007,2022,2022-01-01,0.318692,0.342272
4,1,2,MP 03: Naamsestraat 62 Taste,51.407297,50.094018,50.875900,4.700262,2022,2022-01-02,0.507034,0.542528


In [ ]:
# check for missing values in each column
print(merged.isnull().sum())

month                 0
description           0
lamax                 0
laeq                  0
lat                   0
lon                   0
standardized_lamax    0
standardized_laeq     0
dtype: int64


In [ ]:
# Add initial trace to the figure
fig = px.scatter_mapbox(merged, 
                        lat = 'lat', 
                        lon = 'lon', 
                        size = 'standardized_lamax',
                        size_max = 30,
                        animation_frame="days_since_reference",
                        hover_data={"description":True,"days_since_reference":False,"lat":False,"lon":False,"date": True},
                        zoom = 14, mapbox_style = 'open-street-map')
# Set the initial center and zoom level of the map
fig.update_layout(mapbox={
    'center': {'lat': 50.875900, 'lon': 4.700262}
})

In [ ]:
def marker_click(trace, points, state):
    # Get the clicked marker information
    lat = trace.lat[points.point_inds[0]]
    lon = trace.lon[points.point_inds[0]]
    size = trace.marker.size[points.point_inds[0]]
    text = trace.text[points.point_inds[0]]

    # Perform actions based on the clicked marker
    # For example, display a pop-up or generate a new visualization

# Assign the callback function to the scattermapbox trace
fig.data[0].on_click(marker_click)

# Add dropdown for switching between Lamax and Laeq
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            buttons=list([
                dict(
                    args=[{"marker.size": merged['standardized_lamax']}],
                    label="Lamax",
                    method="restyle"
                ),
                dict(
                    args=[{"marker.size": merged['standardized_laeq']}],
                    label="Laeq",
                    method="restyle"
                )
            ]),
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.3,
            yanchor="top"
        ),
    ]
)

# Show the map
fig.show()